## 7.4 seq2seq 개선

#### 7.4.1 입력데이터 반전

![image](https://github.com/choibigo/Study/assets/38881179/fe1681dc-f09d-43cf-880a-f6911b22fa1d)

- 입력데이터를 반전시켜 학습을 진행하면 정확도도 좋아진다.
- 기울기 전파가 원활해 지기 때문이다.
- 예를 들어 "나는 고양이로소이다"를 "i am a cat"으로 번역하는 문장에서 "나"라는 단어가 "I"로 변환되는 과정을 생각 해보자
- "나"로부터 "I"까지 가려면 "는", "고양이", "로소", "이다"까지 총 4 단어 분량의 LSTM 계층을 거쳐야 한다.
- 따라서 역전파 시 "I"로 부터 전해지는 기울기가 "나"에 도달하기까지, 그 먼 거리만틈 영향을 더 받게 된다. => 입력과 출력 사이의 거리가 짧아 지는 거 아닐까?
- 첫 단어에서는 가 까워져서 학습 효율이 좋아진다고 생각할 수 있지만, 입력 데이터를 반전해도 단어 사이의 '평균'적인 거리는 그대로 이다.

#### 7.4.2 Peeky(엿보기)
- Encoder는 입력 문장을 고정길이 벡터 h로 변환한다.
- 이때 h안에는 Decoder에게 필요한 정보가 담겨 있다. 즉, h가 Decoder에 있어서는 유일한 정보이다.
- 현재의 seq2seq는 최초 시각의 LSTM 계층만이 벡터 h를 이용하고 있다.

![image](https://github.com/choibigo/Study/assets/38881179/fa2d8950-aa42-4611-b26f-22c7548cf1cb)

- 이 중요한 정보를 담은 h를 Decoder의 첫번째 계층 뿐 아니라 다른 층에게도 전달하는 것이다.

![image](https://github.com/choibigo/Study/assets/38881179/bfae0901-ad26-4604-8e50-b8e0b58f91e3)

- 따라서 첫 LSTM만 소유한 정보 h를 여러 계층이 공유할 수 있다.
- LSTM의 입력과 Affine 계층의 입력으로 h가 입력되게 된다.
- Affine 계층으로 입력될때 concat해서 입력되게 된다.

![image](https://github.com/choibigo/Study/assets/38881179/053b4f8d-9e21-44b0-aff4-86c38bbc05c2)

- 추가적으로 Attension이란 기술로 seq2seq를 개선할 수 있다, 이후 Transformer를 통해 성능을 더욱 증가시킬 수 있다.


In [ ]:
class PeekyDecoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(H + D, 4 * H) / np.sqrt(H + D)).astype('f') # LSTM의 입력에 hidden sate의 차원 H 만큼을 추가한다.
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H + H, V) / np.sqrt(H + H)).astype('f') # affine의 입력에 hidden sate의 차원 H 만큼을 추가한다.
        affine_b = np.zeros(V).astype('f')

        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.affine = TimeAffine(affine_W, affine_b)

        self.params, self.grads = [], []
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
        self.cache = None

    def forward(self, xs, h):
        N, T = xs.shape
        N, H = h.shape

        self.lstm.set_state(h)

        out = self.embed.forward(xs)
        hs = np.repeat(h, T, axis=0).reshape(N, T, H) # hidden state릐 T만큼 반복하여 embeding의 output과 동일한 형상을 맞춰준다.
        out = np.concatenate((hs, out), axis=2) # embedding의 output과 concate을 한다.

        out = self.lstm.forward(out)
        out = np.concatenate((hs, out), axis=2) # lstm의 output과 concate한다.

        score = self.affine.forward(out)
        self.cache = H
        return score